In [4]:
#importação de bibliotecas do py
import pandas as pd
import numpy as np

#importação do dataset
df = pd.read_csv("D:/Downloads-UD/dataset-ap-geral.csv", sep=";", encoding="utf-8")

In [8]:
#Ciação do variável preco/m2

if "area(m2)" in df.columns:
    #substitui vírgulas por pontos e converte para numérico (caso necessário)
    df["area"] = df["area(m2)"].astype(str).str.replace(",", ".")
    df["area"] = pd.to_numeric(df["area"], errors="coerce")

    #remove registros com área inválida ou zero para evitar divisões por zero
    df = df[df["area(m2)"].notnull() & (df["area(m2)"] > 0)]

    #calcula o preço por metro quadrado
    df["preco_m2"] = df["valor"] / df["area"]

    #exibe as 5 primeiras linhas
    print(df[["valor", "area(m2)", "preco_m2"]].head(10))
else:
    print("A coluna 'area' não foi encontrada no dataframe.")

     valor  area(m2)      preco_m2
0   753000        84   8964.285714
1  2590000       167  15508.982036
2   579000        54  10722.222222
3   350000        80   4375.000000
4   869000       122   7122.950820
5   950000       182   5219.780220
6   350000        64   5468.750000
7   285000        45   6333.333333
8   550000        70   7857.142857
9  2800000       287   9756.097561


In [9]:
#iqr
Q1 = np.percentile(df["preco_m2"], 25)
Q3 = np.percentile(df["preco_m2"], 75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

#retirada de outliers
outliers = df[(df["preco_m2"] < limite_inferior) | (df["preco_m2"] > limite_superior)]

#stats
print("Estatísticas dos OUTLIERS em preco_m2:\n")
print(outliers["preco_m2"].describe())

#%outliers
percentual = (outliers.shape[0] / df.shape[0]) * 100
print(f"\nOutliers representam {percentual:.2f}% dos imóveis.\n")

#extremos
print("TOP 10 maiores valores de preco_m2 (outliers):")
print(outliers.sort_values("preco_m2", ascending=False)[["valor", "area(m2)", "preco_m2"]].head(10))

Estatísticas dos OUTLIERS em preco_m2:

count      369.000000
mean     19314.178554
std       6320.712662
min      14848.484848
25%      15822.784810
50%      17200.000000
75%      20233.463035
max      79750.000000
Name: preco_m2, dtype: float64

Outliers representam 4.39% dos imóveis.

TOP 10 maiores valores de preco_m2 (outliers):
         valor  area(m2)      preco_m2
576    1595000        20  79750.000000
7056  31677000       570  55573.684211
1586  14000000       270  51851.851852
8136   1967863        48  40997.145833
1921   1967863        48  40997.145833
2594  16450000       415  39638.554217
4272  13000000       343  37900.874636
1898   8400000       228  36842.105263
6793   8000001       220  36363.640909
496     360000        10  36000.000000


In [10]:
#bairros/cidades com mais outliers

if "bairro/cidade" in df.columns:
    print("\nTop bairros/cidades com + outliers de preço_m2:")
    print(outliers["bairro/cidade"].value_counts().head(10))


Top bairros/cidades com + outliers de preço_m2:
bairro/cidade
São Paulo         65
Pinheiros         23
Moema             21
Brooklin          18
Perdizes          18
Vila Mariana      16
Rio de Janeiro    15
Campo Belo        13
Vila Olímpia      11
Paraíso           11
Name: count, dtype: int64


In [11]:
# Não-outliers
nao_outliers = df[(df["preco_m2"] >= limite_inferior) & (df["preco_m2"] <= limite_superior)]

# Selecionar colunas de interesse (ajuste conforme suas colunas reais)
colunas_interesse = ["valor", "area(m2)", "quartos", "garagem", "banheiro", "suite", "condominio"]

# Verificar colunas que existem no DataFrame
colunas_presentes = [col for col in colunas_interesse if col in df.columns]

# MÉDIAS
print("=== MÉDIAS DOS OUTLIERS ===")
print(outliers[colunas_presentes].mean())

print("\n=== MÉDIAS DOS NÃO-OUTLIERS ===")
print(nao_outliers[colunas_presentes].mean())

# DIFERENÇA RELATIVA
print("\n=== DIFERENÇA RELATIVA (%) ===")
diferenca = ((outliers[colunas_presentes].mean() - nao_outliers[colunas_presentes].mean()) / nao_outliers[colunas_presentes].mean()) * 100
print(diferenca.round(2))

=== MÉDIAS DOS OUTLIERS ===
valor         2.570433e+06
area(m2)      1.249268e+02
quartos       2.441734e+00
garagem       1.945799e+00
banheiro      2.739837e+00
suite         3.604336e-01
condominio    1.293263e+03
dtype: float64

=== MÉDIAS DOS NÃO-OUTLIERS ===
valor         546337.225501
area(m2)          79.163741
quartos            2.273565
garagem            1.306811
banheiro           1.817831
suite              0.341551
condominio       718.345412
dtype: float64

=== DIFERENÇA RELATIVA (%) ===
valor         370.48
area(m2)       57.81
quartos         7.40
garagem        48.90
banheiro       50.72
suite           5.53
condominio     80.03
dtype: float64
